In [1]:
# Import packages for use
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import time
import psutil
import cudf
from extract_load_functions import extraction, loading

#set export location
exportLocation = r'/home/jeff/'

#set number of test runs to prefrom
iterations = 31

# Connect to MySQL database
mydb = mysql.connector.connect(host="127.0.0.1",
                               user="root",
                               passwd="0861137MySQL!",
                               database="project1")

print("Your current DB connection is with: ",mydb)


etlTimerStart = time.perf_counter() 
### MEDIUM EXTRACTION ###
##################################################################################################################
# Join the order_line and item tables and capture CPU, RAM and elapsed time to perform operations
sql = "select * from order_line INNER JOIN item ON item.orderedItem =order_line.orderedItem"

# Pull the item table from MySQL
mediumExtraction  = extraction(mydb,sql,iterations,exportLocation,"mediumExtractionPrfm.csv")


### MEDIUM TRANSFORMATION ###
##################################################################################################################
# Preform light transformation workload.  In this case, identify all item descriptions with "Blue" in them 
# and change them to "Navy"

column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
medTransPrfm = pd.DataFrame(columns = column_names)

print("STARTING TRANSFORMATION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

#Run 30 iterations to collect transformation df of performance metrics
transTimerStart= time.perf_counter() 
for sampleNoTransform in range(iterations):
    medTrans = mediumExtraction
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    medTrans.item_desc = medTrans.item_desc.str.replace('Blue', 'Navy', regex=True)
    medTrans['extended_price'] = medTrans.selling_price * medTrans.eaches_qty
    
    #Remove duplicate columns
    medTrans = medTrans.loc[:,~medTrans.columns.duplicated()]
    
    #Stop timer  
    stop = time.perf_counter()
    
    medTransPrfm = medTransPrfm.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)

    time.sleep(5)
transTimerEnd= time.perf_counter() 

print("Pandas medium load transformation metrics captured.")

#Export DF to csv
medTransPrfm.to_csv (r'/home/jeff/medTransPrfm.csv', index = False, header=True)

print("Pandas transformation metrics captured.\n")
print("Total TRANSFORMATION time: ", transTimerEnd - transTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average TRANSFORMATION iteration time: ", medTransPrfm.elapsed_time.mean(), "s")
print("Average TRANSFORMATION CPU utilization: ", medTransPrfm.CPU_utilization.mean())
print("Average TRANSFORMATION RAM utilization: ", medTransPrfm.RAM_utilization.mean(), "\n\n")


### MEDIUM cuDF TRANSFORMATION ###
##################################################################################################################
# Preform light transformation workload.  In this case, identify all item descriptions with "Blue" in them 
# and change them to "Navy"
column_names = ["CPU_utilization", "RAM_utilization", "elapsed_time"]
medTransPrfmCU = pd.DataFrame(columns = column_names)

print("STARTING cuDF TRANSFORMATION")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

#Run 30 iterations to collect transformation df of performance metrics
transCuTimerStart = time.perf_counter() 
for sampleNoTransform in range(iterations):
    ## Transform pandas df to cuDF
    medTransCU = cudf.DataFrame.from_pandas(medTrans)
    
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    medTransCU.item_desc = medTransCU.item_desc.str.replace('Blue', 'Navy', regex=True)
    medTransCU['extended_price'] = medTransCU.selling_price * medTransCU.eaches_qty
    
    #Stop timer  
    stop = time.perf_counter()
    
    
    medTransPrfmCU = medTransPrfmCU.append(pd.DataFrame({'CPU_utilization': psutil.cpu_percent(),
                                                         'RAM_utilization':  psutil.virtual_memory().percent,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)

    
    time.sleep(5)
transCuTimerEnd= time.perf_counter() 

print("cuDF medium load transformation metrics captured.")

#Export cuDF performance to csv
medTransPrfmCU.to_csv (r'/home/jeff/medTransPrfmCU.csv', index = False, header=True)

print("cuDF performance metrics captured loading complete.\n")
print("Total cuDF TRANSFORMATION time: ", transCuTimerEnd - transCuTimerStart, "s")
print("Iterations performed: ", iterations)
print("Average cuDF TRANSFORMATION iteration time: ", medTransPrfmCU.elapsed_time.mean(), "s")
print("Average cuDF TRANSFORMATION CPU utilization: ", medTransPrfmCU.CPU_utilization.mean())
print("Average cuDF TRANSFORMATION RAM utilization: ", medTransPrfmCU.RAM_utilization.mean(),"\n\n")

### MEDIUM LOADING ###
###################################################################################################################

loading(medTrans,'mediumTransformation',iterations, exportLocation, "mediumLoadPrfm.csv")

etlTimerEnd = time.perf_counter() 
print('ETL is complete')
print('Elapsed ETL time is: ', (etlTimerEnd-etlTimerStart)/60, ' minutes')

Your current DB connection is with:  <mysql.connector.connection_cext.CMySQLConnection object at 0x7fa3841cfe50>

STARTING EXTRACTION...
Base EXTRACTION CPU utilization:  6.4
Base EXTRACTION RAM utilization:  15.5
RUNNING...
Data frame loading complete.

Total EXTRACTION time:  84.58833564099768 s
Iterations performed:  31
Average EXTRACTION iteration time:  0.7258469406775886 s
Average CPU utilization:  3.6387096774193552
Average RAM utilization:  16.27096774193549 


STARTING TRANSFORMATION...
Base CPU utilization:  1.7
Base RAM utilization:  16.3
RUNNING...
Pandas medium load transformation metrics captured.
Pandas transformation metrics captured.

Total TRANSFORMATION time:  159.4475968830011 s
Iterations performed:  31
Average TRANSFORMATION iteration time:  0.1322492651623531 s
Average TRANSFORMATION CPU utilization:  1.8645161290322587
Average TRANSFORMATION RAM utilization:  16.300000000000008 


STARTING cuDF TRANSFORMATION
Base CPU utilization:  1.6
Base RAM utilization:  16.